In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from utils.data_preprocess import * # data_split, DataLabeling, train_val_test_split
from models import *

In [2]:
WINDOW_SIZE = 24

In [16]:
df = pd.read_csv('data\ETHBTC-5m-data.csv')
raw_data = DataLabeling(df, WINDOW_SIZE)
train_df, val_df, test_df = train_val_test_split(raw_data.labelled_data, rolling=150000)
Data = DataPreprocess(train_df, val_df, test_df, WINDOW_SIZE, label_size=1, label_columns=['Trend'], shift=0)

In [24]:
model = MLP_model(24, 5)

In [25]:
test_model = model.model

In [26]:
test_model.fit(Data.train,
               epochs=20000,
               validation_data=Data.val,
               steps_per_epoch=len(Data.train),
               validation_steps=int(0.15*len(Data.val)),
               callbacks=[create_model_checkpoint(model_name=test_model.name, save_path='ETHBTC_without_TIs'),
                          tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)])

Epoch 1/20000
11867/11867 [==============================] - 202s 17ms/step - loss: 0.7583 - accuracy: 0.5123 - val_loss: 0.8918 - val_accuracy: 0.5000
Epoch 2/20000
11867/11867 [==============================] - 199s 17ms/step - loss: 0.7350 - accuracy: 0.5163 - val_loss: 0.8908 - val_accuracy: 0.3333
Epoch 3/20000
11867/11867 [==============================] - 193s 16ms/step - loss: 0.7336 - accuracy: 0.5141 - val_loss: 0.8958 - val_accuracy: 0.3333
Epoch 4/20000
11867/11867 [==============================] - 196s 17ms/step - loss: 0.7334 - accuracy: 0.5130 - val_loss: 0.8940 - val_accuracy: 0.3333
Epoch 5/20000
11867/11867 [==============================] - 196s 16ms/step - loss: 0.7316 - accuracy: 0.5120 - val_loss: 0.8913 - val_accuracy: 0.3333
Epoch 6/20000
11867/11867 [==============================] - 198s 17ms/step - loss: 0.7310 - accuracy: 0.5126 - val_loss: 0.8922 - val_accuracy: 0.3333
Epoch 7/20000
11867/11867 [==============================] - 204s 17ms/step - loss: 0.73

In [28]:
test_model = tf.keras.models.load_model('ETHBTC_without_TIs\MLP_model')
test_model.evaluate(Data.test)

7/7 [==============================] - 0s 7ms/step - loss: 0.7050 - accuracy: 0.4658


[0.704985499382019, 0.465753436088562]

In [30]:
tf.argmax(test_model.predict(Data.test), axis=1)

<tf.Tensor: shape=(73,), dtype=int64, numpy=
array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2], dtype=int64)>

In [23]:
test_df.Trend.value_counts()

0    62
2    34
Name: Trend, dtype: int64